In [3]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial
import os
import csv

import pandas as pd
import boto3
import tqdm

In [5]:
df = pd.read_csv("cleaned_11000.csv")
ori = df.copy()

In [9]:
adware_q = df.loc[df['malware_type'] == "adware"]
adware_list = adware_q["sha256"].tolist()
adware_list

['...']

In [15]:
AWS_BUCKET = "sorel-20m"
OUTPUT_DIR = "C:\\Users\\swe19\\Desktop\\mal1\\adware\\"

def download_one_file(bucket: str, output: str, client: boto3.client, s3_file: str):
    """
    Download a single file from S3
    Args:
        bucket (str): S3 bucket where images are hosted
        output (str): Dir to store the images
        client (boto3.client): S3 client
        s3_file (str): S3 object name
        
    print(bucket, s3_file, os.path.join(output, s3_file))
    print("outputname" , output_name)
    """
    s3_dir = "09-DEC-2020/binaries/"
    s3_file = s3_dir+s3_file
    output_file = s3_file.split("/")[-1]
    output_dest = output + output_file
    client.download_file(
        Bucket=bucket, Key=s3_file, Filename=output_dest
    )


files_to_download = adware_list
# Creating only one session and one client
session = boto3.Session()
client = session.client("s3")
# The client is shared between threads
func = partial(download_one_file, AWS_BUCKET, OUTPUT_DIR, client)

# List for storing possible failed downloads to retry later
failed_downloads = []

with tqdm.tqdm(desc="Downloading files from S3", total=len(files_to_download)) as pbar:
    with ThreadPoolExecutor(max_workers=32) as executor:
        # Using a dict for preserving the downloaded file for each future, to store it as a failure if we need that
        futures = {
            executor.submit(func, file_to_download): file_to_download for file_to_download in files_to_download
        }
        for future in as_completed(futures):
            if future.exception():
                failed_downloads.append(futures[future])
            pbar.update(1)
if len(failed_downloads) > 0:
    print("Some downloads have failed. Saving ids to csv")
    with open(
        os.path.join("failed_downloads.csv"), "w", newline=""
    ) as csvfile:
        wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
        wr.writerow(failed_downloads)

Some downloads have failed. Saving ids to csv


In [ ]:
# https://emasquil.github.io/posts/multithreading-boto3/